In [ ]:
 %matplotlib inline

import os, sys
sys.path.extend(["../models", "../utils", "../handle_data"])
import numpy as np
import xarray as xr
import pandas as pd
from statistical_evaluation import Scores
from statistical_evaluation import perform_block_bootstrap_metric as bootstrapping
# for plotting
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
results_base_dir = "../results"
exps_eval = ["unet_new_t2m", "wgan_new_t2m", "deepru_t2m_new"]
nboots = 1000

plt_fname = "rmse_meta_postprocessing.png"

In [ ]:
nexps = len(exps_eval)

rmse_avg = xr.DataArray(np.zeros(nexps), coords={"experiment": exps_eval}, dims="experiment")
rmse_boot = xr.DataArray(np.zeros(nexps*nboots).reshape(nexps, nboots),
                         coords={"experiment": exps_eval, "iboot": np.arange(nboots)},
                         dims=["experiment", "iboot"])

In [ ]:
for i, exp in enumerate(exps_eval):
    ds = xr.open_dataset(os.path.join(results_base_dir, exps_eval[i], "postprocessed_ds_test.nc"))
    score_engine = Scores(ds["t_2m_tar_fcst"], ds["t_2m_tar_ref"], ["rlat", "rlon"])
    rmse_tall_exp = score_engine("rmse")
    rmse_avg[i], rmse_boot[i,:] = rmse_tall_exp.mean(), bootstrapping(rmse_tall_exp, "time", 120)

In [ ]:
fs = 16

# create figure
fig, ax = plt.subplots(1, 1)
# create box-plot
bp = ax.boxplot(rmse_boot.T, labels=["U-Net (Sha)", "WGAN (Sha)", "DeepRU"], patch_artist=True)
# configure plot
ax.set_ylim(0.8, 1.4)

ax.set_title("")
ax.set_ylabel("RMSE", fontsize=fs)
ax.tick_params(axis="both", which="both", direction="out", labelsize=fs-2)

colors = ['pink', 'lightblue', 'lightgreen']
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    
for median in bp['medians']:
    median.set_color('black')
    median.set_linewidth(2.)
    
fig.savefig(plt_fname, bbox_inches="tight")
plt.tight_layout()
fig.savefig(plt_fname)
plt.close(fig)